In [1]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import time
import sys

#Set device
USE_CUDA = torch.cuda.is_available()
if USE_CUDA:
    device = torch.device("cuda")
    cuda = True
else:
    device = torch.device("cpu")
    cuda = False
    
print("Device =", device)
gpus = [0]

Device = cuda


In [2]:
def time_elapsed(start_time):
    elapsed = time.time() - start_time
    hours = int(elapsed/3600)
    minutes = int(int(elapsed/60)%60)
    seconds = int(elapsed%60)
    
    return hours, minutes, seconds

In [3]:
#Load SMILES data as integer labels and as one-hot encoding
data = np.load("ohesmiles.npz")
data = data["arr_0"]

intdata = np.load("intsmiles.npz")
intdata = intdata["arr_0"]

data = torch.from_numpy(data).view(np.shape(data)[0], 1, np.shape(data)[1])
intdata = torch.from_numpy(intdata)

print("Dataset size: " + str(data.size()))
print("Integer dataset size: " + str(intdata.size()))

Dataset size: torch.Size([24825178, 1, 53])
Integer dataset size: torch.Size([24825178])


In [4]:
#Get input tensor
def inptarg(i, batch_size):
    while True:
        try:
            #Define input and target tensor sizes
            inp = torch.Tensor((seq_length - 1) * batch_size, 1, np.shape(data)[2])
            target = torch.Tensor((seq_length - 1) * batch_size)
    
            #SMILES molecules in batch 
            inputs = data[int((i * seq_length)) : int((i * seq_length) + (seq_length * batch_size)), :, :]
            targets = intdata[int((i * seq_length)) : int((i * seq_length) + (seq_length * batch_size))]

            #Index counters for input, target
            r = 0
            s = 0 
            for p in range(seq_length * batch_size - 1):
        
                if (p % seq_length != (seq_length - 1)):
                    #Input data (does not include last character in SMILES)
                    inp[r, :, :] = inputs[p, :, :]
                    r += 1
            
                if (p % seq_length != 0):
                    #Target data (does not include first character in SMILES)
                    target[s] = targets[p]
                    s += 1
                    
            return inp, target
        
        except:
            continue
        break

In [29]:
#Define model
class Model(nn.Module):
    
    #Define model parameters
    def __init__(self, input_size, hidden_size, num_layers, dropout, temperature):
        super(Model, self).__init__()
        
        #Model parameters
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = dropout
        self.temperature = temperature
        
        #Model layers
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, dropout = dropout)
        self.linear = nn.Linear(hidden_size, input_size)
        self.softmax = nn.Softmax(dim=2)
        
    #Define initial hidden and cell states
    def init_states(self, num_layers, hidden_size):
        hidden = [Variable(torch.zeros(num_layers, 1, hidden_size)),
                  Variable(torch.zeros(num_layers, 1, hidden_size))]
        
        return hidden
    
    #Define forward propagation
    def forward(self, inp, hidden, temperature):
        #LSTM
        output, hidden = self.lstm(inp, hidden)

        #Linear Layer
        output = self.linear(output)
        
        #Softmax Layer
        output = self.softmax(output)
        
        return output, hidden

In [30]:
#Define training
def train(epochs):
    #Set start time
    start_time = time.time()
    
    #Does not support shuffling of the data (converges well within an epoch)
    #Iterate over desired number of epochs 
    for e in range(epochs):
        
        #Iterate set of seq_length characters
        for i in range(int((np.shape(data)[0] - seq_length + 1) / batch_size)):
            
            #Initialize hidden and cell states
            hidden = model.init_states(num_layers, hidden_size)
            
            #Run on GPU if available
            if cuda:
                hidden = (hidden[0].cuda(), hidden[1].cuda())
        
            #Set initial gradients
            model.zero_grad()
    
            #Set initial loss
            loss = 0 
            
            #Get input and target
            input_data, target_data = inptarg(i, batch_size)
            input_data = input_data.float()
            target_data = target_data.long()
            
            #Run on GPU if available
            if cuda:
                input_data = input_data.cuda()
                target_data = target_data.cuda()
                
            #Run model, calculate loss
            output, hidden = model(input_data, hidden, temperature)
            loss += criterion(output.squeeze(), target_data.squeeze())
                
            #Backpropagate loss
            loss.backward()
            optimizer.step()
            
            #Update list of losses
            if (e == i == 0):
                losses[0] = loss.data.item() / seq_length   
            losses.append(loss.data.item() / seq_length)
            
            if (i % 1000 == 0):
                torch.save(model.state_dict(), "network.pth")
            
            if(i == 1000):
                torch.save(model.state_dict(), "network1000.pth")
                
            if(i == 2000):
                torch.save(model.state_dict(), "network2000.pth")            
                
            if(i == 3000):
                torch.save(model.state_dict(), "network3000.pth")
                
            if(i == 4000):
                torch.save(model.state_dict(), "network4000.pth")  

            if(i == 6000):
                torch.save(model.state_dict(), "network6000.pth")
                 
            if(i == 8000):
                torch.save(model.state_dict(), "network8000.pth") 
           
            if(i == 10000):
                torch.save(model.state_dict(), "network10000.pth")
            
            if(i == 20000):
                torch.save(model.state_dict(), "network20000.pth")
                
            #Print training info
            hours, minutes, seconds = time_elapsed(start_time)
            print("Loss: {:0.6f}".format(loss.data.item() / seq_length) + " | ΔLossTotal: {:+0.4f}".format(losses[-1] - losses[1]) + " | Epoch: {0:02d}".format(e + 1) + " | Iteration: {0:04d}".format(i + 1) + " | Time elapsed: {0:02d}".format(hours) + "h {0:02d}".format(minutes) + " m {0:02d}".format(seconds) + " s")

In [34]:
#Initialize model parameters
input_size = np.shape(data)[2]
hidden_size = 1024
num_layers = 3
dropout = .2
learning_rate = 0.001
epochs = 1
seq_length = 75
batch_size = 128
temperature = 1

#List of losses
losses = [0]

In [35]:
#Call model, set optimizer and loss function
model = Model(input_size, hidden_size, num_layers, dropout, temperature)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
criterion = nn.CrossEntropyLoss()

#Run on GPU if available
if cuda:
    model.cuda()
    criterion.cuda()  

In [36]:
#Total number of parameters
total_params = sum(p.numel() for p in model.parameters())
print("Total number of parameters in network: " + str(total_params))

Total number of parameters in network: 21267509


In [37]:
#Train
train(epochs)

Loss: 0.052938 | ΔLossTotal: +0.0000 | Epoch: 01 | Iteration: 0001 | Time elapsed: 00h 00 m 01 s
Loss: 0.052920 | ΔLossTotal: -0.0000 | Epoch: 01 | Iteration: 0002 | Time elapsed: 00h 00 m 02 s
Loss: 0.052874 | ΔLossTotal: -0.0001 | Epoch: 01 | Iteration: 0003 | Time elapsed: 00h 00 m 03 s
Loss: 0.052299 | ΔLossTotal: -0.0006 | Epoch: 01 | Iteration: 0004 | Time elapsed: 00h 00 m 04 s
Loss: 0.050792 | ΔLossTotal: -0.0021 | Epoch: 01 | Iteration: 0005 | Time elapsed: 00h 00 m 05 s
Loss: 0.050739 | ΔLossTotal: -0.0022 | Epoch: 01 | Iteration: 0006 | Time elapsed: 00h 00 m 06 s
Loss: 0.050691 | ΔLossTotal: -0.0022 | Epoch: 01 | Iteration: 0007 | Time elapsed: 00h 00 m 07 s
Loss: 0.050680 | ΔLossTotal: -0.0023 | Epoch: 01 | Iteration: 0008 | Time elapsed: 00h 00 m 08 s
Loss: 0.050663 | ΔLossTotal: -0.0023 | Epoch: 01 | Iteration: 0009 | Time elapsed: 00h 00 m 09 s
Loss: 0.050655 | ΔLossTotal: -0.0023 | Epoch: 01 | Iteration: 0010 | Time elapsed: 00h 00 m 10 s
Loss: 0.050647 | ΔLossTotal: -

Loss: 0.050440 | ΔLossTotal: -0.0025 | Epoch: 01 | Iteration: 0086 | Time elapsed: 00h 01 m 32 s
Loss: 0.050412 | ΔLossTotal: -0.0025 | Epoch: 01 | Iteration: 0087 | Time elapsed: 00h 01 m 33 s
Loss: 0.050463 | ΔLossTotal: -0.0025 | Epoch: 01 | Iteration: 0088 | Time elapsed: 00h 01 m 34 s
Loss: 0.050502 | ΔLossTotal: -0.0024 | Epoch: 01 | Iteration: 0089 | Time elapsed: 00h 01 m 35 s
Loss: 0.050480 | ΔLossTotal: -0.0025 | Epoch: 01 | Iteration: 0090 | Time elapsed: 00h 01 m 36 s
Loss: 0.050480 | ΔLossTotal: -0.0025 | Epoch: 01 | Iteration: 0091 | Time elapsed: 00h 01 m 37 s
Loss: 0.050429 | ΔLossTotal: -0.0025 | Epoch: 01 | Iteration: 0092 | Time elapsed: 00h 01 m 38 s
Loss: 0.050418 | ΔLossTotal: -0.0025 | Epoch: 01 | Iteration: 0093 | Time elapsed: 00h 01 m 39 s
Loss: 0.050423 | ΔLossTotal: -0.0025 | Epoch: 01 | Iteration: 0094 | Time elapsed: 00h 01 m 40 s
Loss: 0.050423 | ΔLossTotal: -0.0025 | Epoch: 01 | Iteration: 0095 | Time elapsed: 00h 01 m 42 s
Loss: 0.050429 | ΔLossTotal: -

KeyboardInterrupt: 

In [11]:
torch.save(model.state_dict(), "network.pth")

In [12]:
with open('losses.txt', 'w') as f:
    for item in losses:
        f.write("%s\n" % item)